Some of the data used by `presto` can be acquired from Paituli STAC. Original model uses ERA5 precipitation and temperature, and SRTM elevation and aspect, but here we use data from FMI for precipitation and temperature, and DEM from NLS Finland for elevation and aspect.

First some imports:

In [1]:
import stackstac
import pystac_client
import geopandas as gpd
from pathlib import Path
import os
import xarray as xr
import rioxarray as rxr
import numpy as np
from dask.distributed import Client, Lock
import rasterio as rio

Set URL for Paituli STAC.

In [2]:
paituli_stac_url = 'https://paituli.csc.fi/geoserver/ogc/stac/v1'

Start Dask cluster for faster processing.

In [3]:
client = Client(n_workers=8)

2025-09-16 09:29:36,321 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,345 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,355 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,362 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,367 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,375 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,383 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,394 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,422 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,424 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,428 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,432 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,437 - distributed.nanny - WARNING - Restarting worker
2025-09-16 09:29:36,441 - distributed.

In [4]:
catalog = pystac_client.Client.open(paituli_stac_url)
catalog

<Client id=GeoserverSTACLandingPage>

In [5]:
print(f'There are {len(list(catalog.get_collections()))} collections in Paituli STAC')


There are 177 collections in Paituli STAC


The interesting collections for our purposes are **temperature**, **precipitation**, **slope** and **aspect**, so:

* for precipitation, use [Monthly precipitation sum](https://radiantearth.github.io/stac-browser/#/external/paituli.csc.fi/geoserver/ogc/stac/v1/collections/fmi_monthly_precipitation_10km_at_paituli)
* for temperature, use [Monthly mean temperature](https://radiantearth.github.io/stac-browser/#/external/paituli.csc.fi/geoserver/ogc/stac/v1/collections/fmi_monthly_mean_temperature_10km_at_paituli)
* for slope and aspect, use [MML-DTM-2m_2020](https://radiantearth.github.io/stac-browser/#/external/paituli.csc.fi/geoserver/ogc/stac/v1/collections/2m_digital_terrain_model_products_at_fmi/items/MML-DTM-2m_2020)

## DEM

Filter the collections we need:

In [6]:
for collection in catalog.get_collections(): 
    if 'slope' in collection.id: print(collection.id)

digital_terrain_model_2m_slope_at_geocubes


In [17]:
for collection in catalog.get_collections(): 
    if 'terrain' in collection.id: print(collection.id)

digital_terrain_model_2m_aspect_at_geocubes
digital_terrain_model_2m_slope_at_geocubes
2m_digital_terrain_model_products_at_fmi
digital_terrain_model_10m_at_geocubes
digital_terrain_model_2m_at_geocubes


In [18]:
dtm_collections = ['digital_terrain_model_2m_slope_at_geocubes', 'digital_terrain_model_10m_at_geocubes']

Load area of interest. It corresponds to Sentinel-2 tile 35WNT. Also convert it to `EPSG:4326`.

In [19]:
aoi = gpd.read_file('../data/AOI.geojson').to_crs('epsg:4326')

Search for items within the bounds. First slope.

In [20]:
bounds_epsg_3067 = aoi.to_crs('epsg:3067').total_bounds

In [22]:
slope_search = catalog.search(collections=dtm_collections[0],
)

slope_cube = stackstac.stack(
    items=slope_search.item_collection(),
    epsg=3067, resolution=10
)
slope_cube = slope_cube.sel({'band': '10m'})
del slope_cube.attrs['spec']
slope_cube = slope_cube.sel({'x': slice(bounds_epsg_3067[0], bounds_epsg_3067[2]), 
                         'y': slice(bounds_epsg_3067[3], bounds_epsg_3067[1])
                         })
slope_cube = slope_cube.chunk({'x': 1024, 'y': 1024})
slope_cube = stackstac.mosaic(slope_cube)
slope_cube = slope_cube.to_dataset()
slope_cube = slope_cube.rename({d: 'slope' for d in slope_cube.data_vars})
slope_cube

<xarray.Dataset> Size: 965MB
Dimensions:         (x: 10981, y: 10980)
Coordinates:
    band            <U5 20B '10m'
  * x               (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y               (y) float64 88kB 7.8e+06 7.8e+06 ... 7.69e+06 7.69e+06
    start_datetime  <U29 116B '2022-01-01T00:00:00.000+00:00'
    gsd             int64 8B 10
    proj:code       <U9 36B 'EPSG:3067'
    end_datetime    <U29 116B '2022-12-31T00:00:00.000+00:00'
    title           <U5 20B '10m'
    epsg            int64 8B 3067
Data variables:
    slope           (y, x) float64 965MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>

Then elevation:

In [31]:
height_search = catalog.search(collections=dtm_collections[1],
)

height_cube = stackstac.stack(
    items=height_search.item_collection(),
    epsg=3067, resolution=10
)
height_cube = height_cube.sel({'band': '10m'})
del height_cube.attrs['spec']
height_cube = height_cube.sel({'x': slice(bounds_epsg_3067[0], bounds_epsg_3067[2]), 
                               'y': slice(bounds_epsg_3067[3], bounds_epsg_3067[1])
                               })
height_cube = height_cube.chunk({'x': 1024, 'y': 1024})
height_cube = stackstac.mosaic(height_cube)
height_cube

<xarray.DataArray 'stackstac-28b1731e175bc0a04e0081ac08d15377' (y: 10980,
                                                                x: 10981)> Size: 965MB
dask.array<mosaic-aggregate, shape=(10980, 10981), dtype=float64, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates:
    band            <U5 20B '10m'
  * x               (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y               (y) float64 88kB 7.8e+06 7.8e+06 ... 7.69e+06 7.69e+06
    start_datetime  <U29 116B '2018-01-01T00:00:00.000+00:00'
    gsd             int64 8B 10
    proj:code       <U9 36B 'EPSG:3067'
    end_datetime    <U29 116B '2018-12-31T00:00:00.000+00:00'
    title           <U5 20B '10m'
    epsg            int64 8B 3067
Attributes:
    crs:         epsg:3067
    transform:   | 10.00, 0.00,-13950.00|\n| 0.00,-10.00, 7802190.00|\n| 0.00...
    resolution:  10

Put them into a single dataset and save as `zarr` file.

In [32]:
height_cube = height_cube.to_dataset()
height_cube = height_cube.rename({d: 'elevation' for d in height_cube.data_vars})

In [33]:
height_cube['slope'] = slope_cube['slope']

In [34]:
height_cube

<xarray.Dataset> Size: 2GB
Dimensions:         (x: 10981, y: 10980)
Coordinates:
    band            <U5 20B '10m'
  * x               (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y               (y) float64 88kB 7.8e+06 7.8e+06 ... 7.69e+06 7.69e+06
    start_datetime  <U29 116B '2018-01-01T00:00:00.000+00:00'
    gsd             int64 8B 10
    proj:code       <U9 36B 'EPSG:3067'
    end_datetime    <U29 116B '2018-12-31T00:00:00.000+00:00'
    title           <U5 20B '10m'
    epsg            int64 8B 3067
Data variables:
    elevation       (y, x) float64 965MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    slope           (y, x) float64 965MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>

In [36]:
height_cube.to_zarr('../data/dtm.zarr')

/home/mayrajeo/miniconda3/envs/presto/lib/python3.12/site-packages/zarr/core/dtype/npy/string.py:248: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=29, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  v3_unstable_dtype_warning(self)
/home/mayrajeo/miniconda3/envs/presto/lib/python3.12/site-packages/zarr/core/dtype/npy/string.py:248: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=5, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future ver

## Precipitation and temperature

Because of reasons its far easier to download the precipitation and temperature rasters, and then later stack them to a single dataset than get them via STAC.

In [2]:
prec_files = ['../data/rrmon_2020.tif', '../data/rrmon_2021.tif', '../data/rrmon_2022.tif']
temp_files = ['../data/tmon_2020.tif', '../data/tmon_2021.tif', '../data/tmon_2022.tif']

In [3]:
prec_das = []
temp_das = []

for f in prec_files:
    da = rxr.open_rasterio(f).rename({'band': 'month'})
    da = da.assign_coords(year=int(f.split('_')[1].split('.')[0]))
    da = da.assign_coords(band='precipitation')
    da = da.expand_dims('band')
    da = da.expand_dims('year')
    prec_das.append(da)

for f in temp_files:
    da = rxr.open_rasterio(f).rename({'band': 'month'})
    da = da.assign_coords(year=int(f.split('_')[1].split('.')[0]))
    da = da.assign_coords(band='temperature')
    da = da.expand_dims('band')
    da = da.expand_dims('year')
    temp_das.append(da)

In [4]:
ds = xr.concat([xr.concat(prec_das, dim='year').squeeze('band'),
                xr.concat(temp_das, dim='year').squeeze('band')],
                dim='band')

In [5]:
ds = ds.to_dataset(dim='band')
ds

<xarray.Dataset> Size: 2MB
Dimensions:        (year: 3, month: 12, y: 116, x: 68)
Coordinates:
  * month          (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * x              (x) float64 544B 6.869e+04 7.869e+04 ... 7.287e+05 7.387e+05
  * y              (y) float64 928B 7.782e+06 7.772e+06 ... 6.642e+06 6.632e+06
    spatial_ref    int64 8B 0
  * year           (year) int64 24B 2020 2021 2022
Data variables:
    precipitation  (year, month, y, x) float32 1MB -3.4e+38 ... -3.4e+38
    temperature    (year, month, y, x) float32 1MB -3.4e+38 ... -3.4e+38
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  87.959518432617
    STATISTICS_MEAN:     nan
    STATISTICS_MINIMUM:  22.424102783203
    STATISTICS_STDDEV:   nan
    _FillValue:          -3.4e+38
    scale_factor:        1.0
    add_offset:          0.0

In [6]:
ds.data_vars

Data variables:
    precipitation  (year, month, y, x) float32 1MB -3.4e+38 ... -3.4e+38
    temperature    (year, month, y, x) float32 1MB -3.4e+38 ... -3.4e+38

In [8]:
ds.to_zarr('../data/fmi.zarr')

/home/mayrajeo/miniconda3/envs/presto/lib/python3.12/site-packages/zarr/api/asynchronous.py:233: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
